In [1]:
import os
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [2]:
from data_utils.utils import get_X_y_from_data, data_dict_from_df_tables
from ggmodel_dev.models.landuse.BE2 import model_dictionnary

import pandas as pd
import numpy as np

In [3]:
from ggmodel_dev.graphmodel import GraphModel, concatenate_graph_specs
from ggmodel_dev.validation import score_model, plot_diagnostic, score_model_dictionnary

In [4]:
import os
os.environ["GGGI_db_username"] = 'postgres'
os.environ["GGGI_db_password"] = 'lHeJnnRINyWCzfkDOzKl'
os.environ['GGGI_db_endpoint'] ='database-gggi-1.cg4tog4qy0py.ap-northeast-2.rds.amazonaws.com'
os.environ['GGGI_db_port'] = '5432'

from ggmodel_dev.database import get_variables_df

In [24]:
def prepare_TAI_data():
    data_dict = get_variables_df(['ANPi', 'AYi', 'FPi', 'PTTAi', 'TAi'], exclude_tables=['variabledata', 'foodbalancesheet'])
    data_dict['livestock'] = data_dict['livestock'].drop(columns=['Description_y', 'Unit_y']).rename(columns={'Description_x': 'Description', 'Unit_x': 'Unit'})
    data_dict['foodbalancesheet_new'] = data_dict['foodbalancesheet_new'].query("group == 'animal'")
    
    return data_dict

In [25]:
data_dict = prepare_TAI_data()

emissions TAi: Done
foodbalancesheet FPi: Excluded
foodbalancesheet_new FPi: Done
livestock FPi, ANPi, AYi, PTTAi: Done
variabledata TAi: Excluded


In [27]:
data_dict['emissions']

,ISO,Year,Variable,Item,Value,Description,Unit,table
0,AFG,2001,TAi,Asses,682000.0,Stocks,Head,emissions
1,AFG,2002,TAi,Asses,1588000.0,Stocks,Head,emissions
2,AFG,2005,TAi,Asses,1391000.0,Stocks,Head,emissions
3,AFG,2003,TAi,Asses,1598000.0,Stocks,Head,emissions
4,AFG,2006,TAi,Asses,1215000.0,Stocks,Head,emissions
...,...,...,...,...,...,...,...,...
186596,ZWE,2016,TAi,Chickens,20749000.0,Stocks,Head,emissions
186597,ZWE,2017,TAi,Chickens,21503000.0,Stocks,Head,emissions
186598,ZWE,2018,TAi,Chickens,20149000.0,Stocks,Head,emissions
186599,ZWE,2030,TAi,Chickens,67942996.0,Stocks,Head,emissions


In [ ]:
test = data_dict_from_df_tables([data_dict['livestock']])

In [22]:
data_dict['livestock'].query("Variable == 'FPi'")

,ISO,Item,Description,Year,Unit,Value,Variable,FBS_item,emi_item,table
0,AFG,"Eggs, hen, in shell",Production,1961,tonnes,10000.0,FPi,None,"Chickens, layers",livestock
1,AFG,"Eggs, hen, in shell",Production,1962,tonnes,11000.0,FPi,None,"Chickens, layers",livestock
2,AFG,"Eggs, hen, in shell",Production,1963,tonnes,11500.0,FPi,None,"Chickens, layers",livestock
3,AFG,"Eggs, hen, in shell",Production,1964,tonnes,12000.0,FPi,None,"Chickens, layers",livestock
4,AFG,"Eggs, hen, in shell",Production,1965,tonnes,12800.0,FPi,None,"Chickens, layers",livestock
...,...,...,...,...,...,...,...,...,...,...
259061,TUR,"Snails, not sea",Production,2015,tonnes,481.0,FPi,"Meat, Other",None,livestock
259062,TUR,"Snails, not sea",Production,2016,tonnes,491.0,FPi,"Meat, Other",None,livestock
259063,TUR,"Snails, not sea",Production,2017,tonnes,484.0,FPi,"Meat, Other",None,livestock
259064,TUR,"Snails, not sea",Production,2018,tonnes,479.0,FPi,"Meat, Other",None,livestock


In [101]:
test = data_dict_from_df_tables([data_dict['livestock']])

test['ANPi'] = test['ANPi'].droplevel(['FBS_item', 'table']).dropna()
test['AYi'] = test['AYi'].droplevel(['FBS_item', 'table']).dropna()
test['FPi'] = test['FPi'].droplevel(['FBS_item', 'table']).dropna()
test['PTTAi'] = test['PTTAi'].droplevel(['FBS_item', 'table', 'Item']).rename_axis(index={"emi_item": 'Item'})

test['TAi'] = data_dict['emissions'].set_index(['ISO', 'Year', 'Item'])['Value']

In [111]:
TAi_nodes = {'FPi': {'type': 'input',
                     'unit': '1000 tonnes',
                     'name': 'Food production per food group'},
             'AYi': {'type': 'input',
                     'unit': 'tonnes/head',
                     'name': 'Vector of animal yields'},
             'ANPi': {'type': 'variable',
                      'unit': 'head',
                      'name': 'Animals needed for production per animal type',
                      'computation': lambda FPi, AYi, **kwargs: 1e3 * FPi / AYi
                      },
             'PTTAi': {'type': 'parameter',
                       'unit': '1',
                       'name': 'Production to animal population ratio',
                       },
             'TAi': {'type': 'output',
                     'unit': 'head',
                     'name': 'Animal population',
                     'computation': lambda ANPi, PTTAi, **kwargs: PTTAi * ANPi.groupby(level=['ISO', 'Year', 'emi_item']).sum().rename_axis(index={"emi_item": 'Item'})
                     },
             }

model = GraphModel(TAi_nodes)

In [112]:
res = model.run(test)

In [113]:
scores = score_model(model, test)

In [114]:
scores['score_by_Variable']

,r2,correlation,rmse
Variable,,,
ANPi,1.000000,1.000000,2.056382e+08
TAi,0.999998,0.999999,1.098750e+09
